In [1]:
%%capture --no-stderr
!pip3 install -q --upgrade pip
!pip3 install -q google-cloud-aiplatform
!pip3 install -q langchain
!pip3 install -q langchain-community
!pip3 install -q lxml
!pip3 install -q requests
!pip3 install -q beautifulsoup4
!pip3 install -q unstructured
!pip3 install -q langchain-google-genai
!pip3 install -q google-generativeai
!pip3 install -q tqdm

In [2]:
# restart the kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

# Initial Setup

In [1]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [2]:
# source API key from GCP project and configure genai client
import os
import pathlib
import textwrap
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

key_name = !gcloud services api-keys list --filter="gemini-api-key" --format="value(name)"
key_name = key_name[0]

api_key = !gcloud services api-keys get-key-string $key_name --location="us-central1" --format="value(keyString)"
api_key = api_key[0]

os.environ["GOOGLE_API_KEY"] = api_key

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [3]:
# Define project information
import sys
import subprocess

PROJECT_ID = subprocess.check_output(["gcloud", "config", "get-value", "project"], text=True).strip()
REGION = "us-central1"  # @param {type:"string"}

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: qwiklabs-gcp-00-4079a356a90c


In [4]:
# Set environment vars
BUCKET = f"gs://{PROJECT_ID}/embeddings"
DIMENSIONS=768
DISPLAY_NAME='vertex_docs_qa'
ENDPOINT=f"{REGION}-aiplatform.googleapis.com"
TEXT_GENERATION_MODEL='gemini-pro'
SITEMAP='https://docs.anthropic.com/sitemap.xml'


In [5]:
import os
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

# Task 1: Create Documents from Vertex AI Cloud Documentation Site

## Load and parse sitemap.xml

In [6]:
# Parse the xml of sitemap and get URLs of doc site
import requests
from bs4 import BeautifulSoup

def parse_sitemap(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "xml")
    urls = [element.text for element in soup.find_all("loc")]
    return urls

sites = parse_sitemap(SITEMAP)

In [7]:
# Use this to filter out docs that don't have a corresponding reference page
sites_filtered = [url for url in sites if '/en/docs' in url]

In [8]:
len(sites_filtered)

33

## Load documentation pages using the LangChain UnstructuredURLLoader

In [9]:
# This step will take a few minutes to complete
# you will see download messages below the cell after execution
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=sites_filtered)
documents = loader.load();

In [10]:
to_markdown(documents[1].page_content + "\n\nSource: " + documents[1].metadata["source"])

> Anthropic home page
> 
> Talk to Claude
> 
> Research
> 
> News
> 
> Talk to Claude
> 
> Search
> 
> Navigation
> 
> Use cases
> 
> Classification
> 
> User Guides
> 
> API Reference
> 
> Prompt Library
> 
> Release Notes
> 
> Build with Claude Contest
> 
> Developer Console
> 
> Developer Discord
> 
> Support
> 
> Get started
> 
> Overview
> 
> Quickstart
> 
> Intro to Claude
> 
> Learn about Claude
> 
> Use casesOverviewClassificationContent moderationTicket routing
> 
> Models
> 
> Security and compliance
> 
> Build with Claude
> 
> Define success criteria
> 
> Develop test cases
> 
> Prompt engineering
> 
> Text generation
> 
> Embeddings
> 
> Google Sheets add-on
> 
> Vision
> 
> Tool use (function calling)
> 
> Test and evaluate
> 
> Strengthen guardrails
> 
> Using the Evaluation Tool
> 
> Resources
> 
> Glossary
> 
> System status
> 
> Claude 3 model card
> 
> Anthropic Cookbook
> 
> Anthropic Courses
> 
> Use cases
> 
> Classification
> 
> Claude excels at processing, understanding, and recognizing patterns in text, images, and data. These capabilities make Claude especially powerful for classification tasks.
> 
> This guide walks through the process of determining the best approach for building a classifier with Claude and the essentials of end-to-end deployment for a Claude classifier, from use case exploration to back-end integration.
> 
> Visit our 
> 
> classification cookbooks to see example classification implementations using Claude.
> 
> ​When to use Claude for classification
> 
> When should you consider using an LLM instead of a traditional ML approach for your classification tasks? Here are some key indicators:
> 
> Rule-based classes: Use Claude when classes are defined by conditions rather than examples, as it can understand underlying rules.
> 
> Evolving classes: Claude adapts well to new or changing domains with emerging classes and shifting boundaries.
> 
> Unstructured inputs: Claude can handle large volumes of unstructured text inputs of varying lengths.
> 
> Limited labeled examples: With few-shot learning capabilities, Claude learns accurately from limited labeled training data.
> 
> Reasoning Requirements: Claude excels at classification tasks requiring semantic understanding, context, and higher-level reasoning.
> 
> ​Establish your classification use case
> 
> Below is a non-exhaustive list of common classification use cases where Claude excels by industry.
> 
> Tech & IT
> 
> Content moderation: automatically identify and flag inappropriate, offensive, or harmful content in user-generated text, images, or videos.
> 
> Bug prioritization: calassify software bug reports based on their severity, impact, or complexity to prioritize development efforts and allocate resources effectively.
> 
> Customer Service
> 
> Intent analysis: determine what the user wants to achieve or what action they want the system to perform based on their text inputs.
> 
> Support ticket routing: analyze customer interactions, such as call center transcripts or support tickets, to route issues to the appropriate teams, prioritize critical cases, and identify recurring problems for proactive resolution.
> 
> Healthcare
> 
> Patient triaging: classify customer intake conversations and data according to the urgency, topic, or required expertise for efficient triaging.
> 
> Clinical trial screening: analyze patient data and medical records to identify and categorize eligible participants based on specified inclusion and exclusion criteria.
> 
> Finance
> 
> Fraud detection: identify suspicious patterns or anomalies in financial transactions, insurance claims, or user behavior to prevent and mitigate fraudulent activities.
> 
> Credit risk assessment: classify loan applicants based on their creditworthiness into risk categories to automate credit decisions and optimize lending processes.
> 
> Legal
> 
> Legal document categorization: classify legal documents, such as pleadings, motions, briefs, or memoranda, based on their document type, purpose, or relevance to specific cases or clients.
> 
> ​Implement Claude for classification
> 
> The three key model decision factors are: intelligence, latency, and price.
> 
> For classification, a smaller model like Claude 3 Haiku is typically ideal due to its speed and efficiency. Though, for classification tasks where specialized knowledge or complex reasoning is required, Sonnet or Opus may be a better choice. Learn more about how Opus, Sonnet, and Haiku compare here.
> 
> Use evaluations to gauge whether a Claude model is performing well enough to launch into production.
> 
> ​1. Build a strong input prompt
> 
> While Claude offers high-level baseline performance out of the box, a strong input prompt helps get the best results.
> 
> For a generic classifier that you can adapt to your specific use case, copy the starter prompt below:
> 
> Starter prompt
> 
> We also provide a wide range of prompts to get you started in our prompt library, including prompts for a number of classification use cases, including:
> 
> Sentiment AnalysisDetect the tone and sentiment behind tweets. Understand user emotions, opinions, and reactions in real-time.
> 
> Customer Review ClassificationCategorize feedback into pre-specified tags. Streamline product insights and customer service responses.
> 
> ​2. Develop your test cases
> 
> To run your classification evaluation, you will need test cases to run it on. Take a look at our guide to developing test cases.
> 
> ​3. Run your eval
> 
> ​Evaluation metrics
> 
> Some success metrics to consider evaluating Claude’s performance on a classification task include:
> 
> Criteria Description Accuracy The model’s output exactly matches the golden answer or correctly classifies the input according to the task’s requirements. This is typically calculated as (Number of correct predictions) / (Overall number of predictions). F1 Score The model’s output optimally balances precision and recall. Consistency The model’s output is consistent with its predictions for similar inputs or follows a logical pattern. Structure The model’s output follows the expected format or structure, making it easy to parse and interpret. For example, many classifiers are expected to output JSON format. Speed The model provides a response within the acceptable time limit or latency threshold for the task. Bias and Fairness If classifying data about people, is it important that the model does not demonstrate any biases based on gender, ethnicity, or other characteristics that would lead to its misclassification.
> 
> ​Deploy your classifier
> 
> To see code examples of how to use Claude for classification, check out the Classification Guide in the Anthropic Cookbook.
> 
> Overview
> 
> Content moderation
> 
> linkedin
> 
> On this page
> 
> When to use Claude for classification
> 
> Establish your classification use case
> 
> Implement Claude for classification
> 
> 1. Build a strong input prompt
> 
> 2. Develop your test cases
> 
> 3. Run your eval
> 
> Evaluation metrics
> 
> Deploy your classifier
> 
> Source: https://docs.anthropic.com/en/docs/about-claude/use-cases/classification

In [11]:
len(documents)

33

## Create Document chunks 

In [12]:
# recursively loop through the text and create document chunks for embedding
import warnings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    #separator = "\n",
    chunk_size = 2000,
    chunk_overlap  = 100)

document_chunks = text_splitter.split_documents(documents)

print(f"Number documents {len(documents)}")
print(f"Number chunks {len(document_chunks)}")

document_chunks=[f"content: {chunk.page_content}, source: {chunk.metadata['source']}" for chunk in document_chunks]

Number documents 33
Number chunks 190


# Task 2: Generate embeddings from Document chunks

In [13]:
# make a documents directory
!rm -rf ./documents
!mkdir ./documents

In [14]:
# view the document chunks in a dataframe
import pandas as pd

df = pd.DataFrame(document_chunks, columns =['text'])
df

,text
0,content: Anthropic home page\n\nTalk to Claude...
1,content: Model Anthropic API AWS Bedrock GCP V...
2,content: ​Prompt and output performance\n\nThe...
3,content: Claude 2.1 Claude 2 Claude Instant 1....
4,content: Ticket routing\n\nSecurity and compli...
...,...
185,content: Set appropriate output limits: Use th...
186,content: Anthropic home page\n\nTalk to Claude...
187,content: Example: Safeguarding proprietary ana...
188,content: Anthropic home page\n\nTalk to Claude...


In [15]:
# Run this cell to generate the embeddings files you will later upload to Cloud Storage
from tqdm import tqdm
import json

index_embeddings = []
model = "models/embedding-001"

for index, doc in tqdm(df.iterrows(), total=len(df), position=0):

    response = genai.embed_content(model=model, content=doc['text'], task_type="retrieval_query")

    doc_id=f"{index}.txt"
    embedding_dict = {
        "id": doc_id,
        "embedding": response["embedding"],
    }
    index_embeddings.append(json.dumps(embedding_dict) + "\n")
    
    with open(f"documents/{doc_id}", "w") as document:
          document.write(doc['text'])
    
with open("embeddings.json", "w") as f:
    f.writelines(index_embeddings)

100%|██████████| 190/190 [00:26<00:00,  7.24it/s]


In [16]:
from google.cloud import storage

source_file = '/home/jupyter/embeddings.json'
destination_blob_name = 'embeddings/embeddings.json' # Adjust if needed

client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(PROJECT_ID)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file)

In [17]:
# Upload the embedding files to Cloud Storage
# This step will take a few minutes to complete
import subprocess
gsutil_command = f"gsutil -q cp -r './documents' gs://{PROJECT_ID}/documents"

subprocess.run(['gsutil', '-q', 'cp', '-r', './documents', f'gs://{PROJECT_ID}/documents'])

CompletedProcess(args=['gsutil', '-q', 'cp', '-r', './documents', 'gs://qwiklabs-gcp-00-4079a356a90c/documents'], returncode=0)

# Task 3. Create a Vertex AI Vector Store index

In [18]:
# Create the Vertex AI Vector Search index
# This step will take several minutes to complete
# Wait for this cell to complete before proceeding
index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
      display_name="vertex_docs",
      contents_delta_uri=f"gs://{PROJECT_ID}/embeddings",
      dimensions=768,
      approximate_neighbors_count=150,
      distance_measure_type="DOT_PRODUCT_DISTANCE"
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/546645938624/locations/us-central1/indexes/4158805975039475712/operations/2653598505351774208
MatchingEngineIndex created. Resource name: projects/546645938624/locations/us-central1/indexes/4158805975039475712
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/546645938624/locations/us-central1/indexes/4158805975039475712')


In [19]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="vertex_docs",
    description="Embeddings for the documentation curated from the sitemap.",
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784/operations/9013807079105757184
MatchingEngineIndexEndpoint created. Resource name: projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784')


In [20]:
print("subscribe to techcps")

subscribe to techcps


In [21]:
# This step will take up to 20 minutes to complete
# You can view the deployment in the Vertex AI console on the "Vector Search" tab
# Wait for this cell to complete before proceeding
index_endpoint = index_endpoint.deploy_index(
    index=index, deployed_index_id="vertex_index_deployment"
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784/operations/410805890921267200
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784


In [22]:
INDEX_RESOURCE_NAME=index.resource_name
index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

deployed_index = index.deployed_indexes
deployed_index

[index_endpoint: "projects/546645938624/locations/us-central1/indexEndpoints/2494444437749366784"
deployed_index_id: "vertex_index_deployment"
]

# Task 4: Search Vector Store, add result as context to a query (without using a LangChain Chain)

In [23]:
# In the next cells you will query the model directly using the Vertex AI python SDK
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores.matching_engine import MatchingEngine
from langchain.agents import Tool

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def search_vector_store(question):

    vector_store = MatchingEngine.from_components(
                        index_id=INDEX_RESOURCE_NAME,
                        region=REGION,
                        embedding=embeddings,
                        project_id=PROJECT_ID,
                        endpoint_id=deployed_index[0].index_endpoint,
                        gcs_bucket_name=f"{PROJECT_ID}")
    
    relevant_documentation=vector_store.similarity_search(question, k=8)
    context = "\n".join([doc.page_content for doc in relevant_documentation])[:10000]
    return str(context)

In [24]:
from vertexai.preview.generative_models import GenerativeModel
import warnings

# filter warnings for unused libs
warnings.filterwarnings('ignore')

def ask_question(question):
    context = search_vector_store(question)

    prompt=f"""
        Follow exactly those 3 steps:
        1. Read the context below and aggregrate this data
        Context : {context}
        2. Answer the question using only this context
        3. Show the source for your answers
        User Question: {question}


        If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
        """

    model = GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)

    return to_markdown(f"Question: \n{question} \n\n Response: \n {response.text}")

In [25]:
ask_question("How do I reduce prompt leaks?")

> Question: 
> How do I reduce prompt leaks? 
> 
>  Response: 
>  ##  Summary of Anthropic's documentation on reducing prompt leaks in Claude
> 
> **Prompt leaks** occur when sensitive information intended to be hidden in a prompt is exposed in the model's output. While completely eliminating this risk is difficult, Anthropic offers various strategies to significantly reduce it.
> 
> **Here's a summary of their recommendations:**
> 
> **Before you try to reduce prompt leak:**
> 
> * Use leak-resistant techniques only when necessary, as they can negatively impact performance.
> * Implement leak-resistant techniques only after testing your prompts thoroughly.
> * Monitor outputs for potential leaks using tools like output screening or post-processing.
> 
> **Strategies to reduce prompt leak:**
> 
> * **Separate context from queries:** Use system prompts to isolate key information and context from user queries. 
> * **Use post-processing:** Filter Claude's outputs for keywords that might indicate a leak. Regular expressions, keyword filtering, and other text processing methods can be used for this.
> * **Avoid unnecessary proprietary details:** Don't include information in the prompt that Claude doesn't need to perform the task.
> * **Regular audits:** Periodically review your prompts and Claude's outputs for potential leaks.
> 
> **Additional resources:**
> 
> * Advanced: Chain safeguards: https://docs.anthropic.com/en/docs/test-and-evaluate/strengthen-guardrails/mitigate-jailbreaks
> * Prompt engineering tutorial: https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview
> 
> 
> ## Answer to the user question: "How do I reduce prompt leaks?"
> 
> Based on Anthropic's documentation, here are some ways to reduce prompt leaks:
> 
> * **Separate context from queries:** Instead of including all relevant information in the user prompt, use system prompts to provide additional context and instructions.
> * **Use post-processing:** Filter the model's output for keywords or patterns that might indicate a leak.
> * **Avoid including unnecessary details:** Don't include information in the prompt that Claude doesn't need to complete the task.
> * **Regularly audit your prompts and outputs:** Look for potential leaks in your prompts and the model's outputs, and refine your strategies as needed.
> 
> Remember, reducing prompt leaks is an ongoing process. Regularly reviewing your prompts and outputs and using appropriate mitigation strategies will help you keep your data safe.
> 
> ##  Source for my answer:
> 
> * Anthropic documentation on prompt leaks: https://docs.anthropic.com/en/docs/test-and-evaluate/strengthen-guardrails/reduce-prompt-leak
> 
> 
> I hope this information is helpful!

In [26]:
ask_question("What use cases and capabilities does Anthropic support?")

> Question: 
> What use cases and capabilities does Anthropic support? 
> 
>  Response: 
>  ## Answer based on aggregated data:
> 
> **Use cases:**
> 
> * **Text and code generation:** 
>     * summarize text, answer questions, extract data, translate text, 
>     * explain and generate code, 
>     * create production-level code and operate within complex codebases.
> * **Vision:**
>     * process and analyze visual input,
>     * generate code from images with code snippets or templates based on diagrams,
>     * describe an image for a user with low vision.
> * **Tool use:**
>     * interact with external client-side tools and functions,
>     * reason, plan, and execute actions by generating structured outputs through API calls.
> 
> **Capabilities:**
> 
> * **Text generation:**
>     * Adhere to brand voice for excellent customer-facing experiences such as copywriting and chatbots.
> * **Code generation:**
>     * Create production-level code and operate (in-line code generation, debugging, and conversational querying) within complex codebases.
> * **Translation:**
>     * Build automatic translation features between languages.
> * **Financial forecasting:**
>     * Conduct complex financial forecasts.
> * **Legal analysis:**
>     * Support legal use cases that require high-quality technical analysis, long context windows for processing detailed documents, and fast outputs.
> * **Vision:**
>     * Process and analyze visual input, such as extracting insights from charts and graphs.
> * **Tool use:**
>     * Interact with external client-side tools and functions, allowing Claude to reason, plan, and execute actions by generating structured outputs through API calls.
> 
> **Sources:**
> 
> * https://docs.anthropic.com/en/docs/intro-to-claude
> * https://docs.anthropic.com/en/docs/welcome
> 
> ---
> 
> **Note:** This is just a summary of the information available in the context. For a more comprehensive understanding of Anthropic's use cases and capabilities, please refer to the source documents.

# Task 5: Create Retrieval Augmentation Generation application using LangChain

In [27]:
# To answer questions and chain together the prompt, vector search, returned context and model input use a LangChain "Chain"
# In this case you will use the RetrievalQA chain which is commonly used for Question/Answering applications
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# initialize model using chat
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.0, convert_system_message_to_human=True)

In [28]:
from langchain.prompts import PromptTemplate

template = """
    Follow exactly those 3 steps:
    1. Read the context below and aggregrate this data
    Context : {context}
    
    2. Answer the question using only this context
    3. Show the source for your answers
    User Question: {question}

    If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
    """

prompt = PromptTemplate(input_variables=["context",  "question"], template=template)

In [29]:
from langchain.vectorstores.matching_engine import MatchingEngine

vector_store = MatchingEngine.from_components(
    index_id=INDEX_RESOURCE_NAME,
    region=REGION,
    embedding=embeddings,
    project_id=PROJECT_ID,
    endpoint_id=deployed_index[0].index_endpoint,
    gcs_bucket_name=f"{PROJECT_ID}"
)

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 1}
)

# Test the retriever with a simple search performed above
to_markdown(retriever.get_relevant_documents("How do I get started with Anthropic?")[0].page_content)

> content: Use Workbench to create evals, draft prompts, and iteratively refine based on test results.
> 
> Deploy polished prompts and monitor real-world performance for further refinement.
> 
> Implement Claude
> 
> Set up your environment, integrate Claude with your systems (APIs, databases, UIs), and define human-in-the-loop requirements.
> 
> Test your system
> 
> Conduct red teaming for potential misuse and A/B test improvements.
> 
> Deploy to production
> 
> Once your application runs smoothly end-to-end, deploy to production.
> 
> Monitor and improve
> 
> Monitor performance and effectiveness to make ongoing improvements.
> 
> ​Start building with Claude
> 
> When you’re ready, start building with Claude:
> 
> Follow the Quickstart to make your first API call
> 
> Check out the API Reference
> 
> Explore the Prompt Library for example prompts
> 
> Experiment and start building with the Workbench
> 
> Check out the Anthropic Cookbook for working code examples
> 
> Quickstart
> 
> Overview
> 
> linkedin
> 
> On this page
> 
> What you can do with Claude
> 
> Model options
> 
> Claude 3.5 Family
> 
> Claude 3 Family
> 
> Enterprise considerations
> 
> Implementing Claude
> 
> Start building with Claude, source: https://docs.anthropic.com/en/docs/intro-to-claude

In [30]:
chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [31]:
def ask_question(question: str):
    response = qa({"query": question})

    # since k is set to 1 only return the first source retrieved
    source = response['source_documents']
    
    return to_markdown(f"Response: \n\n {response['result']}")

In [32]:
# Note: You will see a library warning when running this step
ask_question("How do I get started with Anthropic?")

> Response: 
> 
>  1. Follow the Quickstart to make your first API call
> 2. Check out the API Reference
> 3. Explore the Prompt Library for example prompts
> 4. Experiment and start building with the Workbench
> 5. Check out the Anthropic Cookbook for working code examples
> 
> Source: https://docs.anthropic.com/en/docs/intro-to-claude